In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
import copy
import os
from sklearn.model_selection import train_test_split

In [2]:
#just use one csv as an example
filepath = "/wsu/home/groups/eeg/eeg-data/csv/eeg*.csv"
files = glob.glob(filepath)
file = files[0]
for file in sorted(files):
    print(file.split("/")[-1])

eeg1.csv
eeg10.csv
eeg11.csv
eeg12.csv
eeg13.csv
eeg14.csv
eeg15.csv
eeg16.csv
eeg17.csv
eeg18.csv
eeg19.csv
eeg2.csv
eeg20.csv
eeg21.csv
eeg22.csv
eeg23.csv
eeg24.csv
eeg25.csv
eeg26.csv
eeg27.csv
eeg28.csv
eeg29.csv
eeg3.csv
eeg30.csv
eeg31.csv
eeg32.csv
eeg33.csv
eeg34.csv
eeg35.csv
eeg36.csv
eeg37.csv
eeg38.csv
eeg39.csv
eeg4.csv
eeg40.csv
eeg41.csv
eeg42.csv
eeg43.csv
eeg44.csv
eeg45.csv
eeg46.csv
eeg47.csv
eeg48.csv
eeg49.csv
eeg5.csv
eeg50.csv
eeg51.csv
eeg52.csv
eeg53.csv
eeg54.csv
eeg55.csv
eeg56.csv
eeg57.csv
eeg58.csv
eeg59.csv
eeg6.csv
eeg60.csv
eeg61.csv
eeg62.csv
eeg63.csv
eeg64.csv
eeg65.csv
eeg66.csv
eeg67.csv
eeg68.csv
eeg69.csv
eeg7.csv
eeg70.csv
eeg71.csv
eeg72.csv
eeg73.csv
eeg74.csv
eeg75.csv
eeg76.csv
eeg77.csv
eeg78.csv
eeg79.csv
eeg8.csv
eeg9.csv


In [4]:
#get column information
df = pd.read_csv(file)

df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace("-ref", "")
df = df.reindex(sorted(df.columns), axis=1)
print(df.columns)

info_cols = ["time", "label_0", "label_1", "label_2", "resp effort", "ecg ekg"]
label_col = ["label_0", "label_1", "label_2"]
data_cols = list(set(df.columns).difference(info_cols))
cols = df[data_cols].columns

Index(['ecg ekg', 'eeg c3', 'eeg c4', 'eeg cz', 'eeg f3', 'eeg f4', 'eeg f7',
       'eeg f8', 'eeg fp1', 'eeg fp2', 'eeg fz', 'eeg o1', 'eeg o2', 'eeg p3',
       'eeg p4', 'eeg pz', 'eeg t3', 'eeg t4', 'eeg t5', 'eeg t6', 'label_0',
       'label_1', 'label_2', 'resp effort', 'time'],
      dtype='object')


In [15]:
# df2 = pd.read_csv(files[4])
df2.columns = df2.columns.str.lower()
df2.columns = df2.columns.str.replace("-ref", "")
df2 = df2.reindex(sorted(df.columns), axis=1)
print(df2.columns)

Index(['ecg ekg', 'eeg c3', 'eeg c4', 'eeg cz', 'eeg f3', 'eeg f4', 'eeg f7',
       'eeg f8', 'eeg fp1', 'eeg fp2', 'eeg fz', 'eeg o1', 'eeg o2', 'eeg p3',
       'eeg p4', 'eeg pz', 'eeg t3', 'eeg t4', 'eeg t5', 'eeg t6', 'label_0',
       'label_1', 'label_2', 'resp effort', 'time'],
      dtype='object')


In [13]:
print(list(set(df.columns).difference(df2.columns)))
print(sorted(data_cols))

[]
['eeg c3', 'eeg c4', 'eeg cz', 'eeg f3', 'eeg f4', 'eeg f7', 'eeg f8', 'eeg fp1', 'eeg fp2', 'eeg fz', 'eeg o1', 'eeg o2', 'eeg p3', 'eeg p4', 'eeg pz', 'eeg t3', 'eeg t4', 'eeg t5', 'eeg t6']


In [5]:
#function to process the csv files and extract the labeled sequences
counter = 0
def process_data(file):
    global counter
    try:
        #load dataframe and set index
        df = pd.read_csv(file)
        df.columns = df.columns.str.lower()
        df.columns = df.columns.str.replace("-ref", "")
        df = df.reindex(sorted(df.columns), axis=1)
        
#         df.set_index("time", inplace=True)
#         df.index = df.index.astype("timedelta64[ns]")
        print(df.columns)

        #extract numpy arrays
        yt = df[label_col].values

        #find find where all annotations agree and make new column for label
        labels = []
        for row in yt:
            if sum(row) == 3:
                labels.append(1)
            else:
                labels.append(0)
        df['label'] = np.array(labels)

        Xt = df[data_cols].values
        #break up into 1 second chunks
        X = [Xt[i:i + 256] for i in range(0, len(Xt), 256)]
        y = df.label.values
        
        #extract discrete sequences of data labeled as 1s and 0s
        state = 0
        seq = []
        master = []
        labels = []
        bdf = pd.DataFrame(columns=data_cols)
        for feature,label in zip(X, y):
            labels.append(label)
            if len(feature) == 256:
                df = pd.DataFrame(feature, columns=data_cols)
                print(df.head())
                df['label'] = label
                df['id'] = counter
                bdf = pd.concat([df, bdf], join="inner", ignore_index=True)
                counter += 1
        return bdf
    except Exception as e:
        print(str("ERROR WITH FILE {}, {}").format(file, e))
        

In [ ]:
#test with one file
df = process_data(files[0])
print(df.head())
print(df.tail())

Index(['ecg ekg', 'eeg c3', 'eeg c4', 'eeg cz', 'eeg f3', 'eeg f4', 'eeg f7',
       'eeg f8', 'eeg fp1', 'eeg fp2', 'eeg fz', 'eeg o1', 'eeg o2', 'eeg p3',
       'eeg p4', 'eeg pz', 'eeg t3', 'eeg t4', 'eeg t5', 'eeg t6', 'label_0',
       'label_1', 'label_2', 'resp effort', 'time'],
      dtype='object')


In [ ]:
#process all the files and concatenate the result to a master array
df = pd.DataFrame(columns=data_cols)
for f in files:
    try:
        print(f)
        bdf = process_data(f)
        pd.concat([df, bdf], axis=1)
    except TypeError:
        print("error with file ", f)

/wsu/home/groups/eeg/eeg-data/csv/eeg1.csv


In [61]:
#split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [65]:
#save numpy array to disk
np.save("data/X_train.npy", X_train)
np.save("data/y_train.npy", y_train)
np.save("data/X_test.npy", X_test)
np.save("data/y_test.npy", y_test)

In [ ]:
pd.DataFrame(X_train).to_csv("data/X_train.csv")
pd.DataFrame(y_train).to_csv("data/y_train.csv")
pd.DataFrame(X_test).to_csv("data/X_test.csv")
pd.DataFrame(y_test).to_csv("data/y_test.csv")

In [38]:
print(counter)

Empty DataFrame
Columns: [eeg c3, eeg f8, eeg f4, eeg cz, eeg pz, eeg f3, eeg p4, eeg fp2, eeg o1, eeg t6, eeg o2, eeg fp1, eeg fz, eeg t3, eeg f7, eeg c4, eeg t5, eeg t4, eeg p3]
Index: []


In [ ]:
#function to process the csv files and extract the labeled sequences
def process_data(file, counter):
    try:
        #load dataframe and set index
        df = pd.read_csv(file)
        df.columns = df.columns.str.lower()
        df.columns = df.columns.str.replace("-ref", "")
        
#         df.set_index("time", inplace=True)
#         df.index = df.index.astype("timedelta64[ns]")
        print(df.columns)

        #extract numpy arrays
        X = df[data_cols].values
        y = df[label_col].values

        #find find where all annotations agree and make new column for label
        labels = []
        for row in y:
            if sum(row) == 3:
                labels.append(1)
            else:
                labels.append(0)
        df['label'] = np.array(labels)

        X = df[data_cols].values
        y = df.label.values
        df.drop(columns=info_cols, inplace=True)
        print(len(y))
        
        #extract discrete sequences of data labeled as 1s and 0s
        state = 0
        seq = []
        master = []
        labels = []
        for feature,label in zip(X,y):
            if label==state:
                seq.append(feature)
            else:
                seconds = len(seq)//1000
                if seconds>=10:
                    #split array into 10 second chunks
                    X,y = [],[]
                    for feature in seq:
                        x_ = np.array_split(feature, 256) #1 second chunks
                        for elm in x_:
                            df = pd.DataFrame(seq, columns=data_cols)
                            df['label'] = state
                            df.to_csv("data/dataset.csv", mode='a', header=(not os.path.exists(output_path)))
                            counter +=1
                    state = label
                    seq = []
        return counter
    except Exception as e:
        print(str("ERROR WITH FILE {}, {}").format(file, e))
        